In [ ]:



#preparing datasets
!pip install transformers datasets
#!conda install pytorch==1.7.0 torchvision==0.8.1 torchaudio==0.7.0 -c pytorch

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset,  load_metric
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification

raw_datasets = load_dataset("imdb")

#The raw_datasets object is a dictionary with three keys:
# "train", "test" and "unsupervised"

#for our purpuse we will use BERT dictionary/model
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tk_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation = True)

tokenized_datasets = raw_datasets.map(tk_function, batched=True)

#before using pytorch, we need to modify our tokenized datasets according to Pytorch requirements
#1. we remove the column "text" (column corresponding to values the model does not expect) - we only keep the tokenized values
#2. we rename the "label" column to "labels"
#3. change the format of the datasets from lists to TENSORS

torch.cuda.empyt_cache()
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")





In [ ]:
data_collator = DataCollatorWithPadding(tokenizer)

train_dataloader = DataLoader(
  tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
  tokenized_datasets["test"], batch_size=8, collate_fn=data_collator
)

#DISCLAIMER !!!! - I did not use the below code from the tutorial, because it reporeted different errors ('label', ...)
#Instead I used the above way of declaring train & eval dataloaders
#This is the code i did NOT use (as mentioned):
  #small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
  #small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
  #now we can define our dataloader class. It represents a Python iterable over a dataset, 
  #with support for map-style and iterable-style datasets, customizing data loading order,
  #automatic batching, single- and multi-process data loading, automatic memory pinning.
  #train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
  #eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)


In [ ]:
# again we define our model

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

# now the only two things missing are an optimizer and a learning rate scheduler
# AdamW is a default optimizer used by Traniner, where lr is the learning rate

from transformers import AdamW
from tranformers import get_scheduler

# to decrease trainig time  use a bigger lr value (=5e-2)
optimizer = AdamW(model.parameters(), lr=5e-5)

# num_epochs indicates how many times the .train will use all of the training data
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps #defined above
)


#to use our GPU to train our model we define a device as follows:

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)


In [ ]:
# we can use tqdm library to display a progress bar

from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))




In [ ]:
#Training the model + explanation

#loss.backwards():
    #loss.backward() computes dloss/dx for every parameter x which has requires_grad=True. These are accumulated into x.grad for every parameter x. In pseudo-code:
    #x.grad += dloss/dx
#optimizer.step()
    #optimizer.step updates the value of x using the gradient x.grad. For example, the SGD optimizer performs:
    #x += -lr * x.grad
#optimizer.zero_grad() clears x.grad for every parameter x in the optimizer. It’s important to call this before loss.backward(), otherwise you’ll accumulate the gradients from multiple passes.#

# Adam optimizer is used: both momentum and adaptive learning rate are used for better convergence.
# In this, the model parameters are altered after computation of loss on each training example. 
#So, if the dataset contains 1000 rows SGD will update the model parameters 1000 times in one cycle of dataset instead of one time as in Gradient Descent.
  #θ=θ−α⋅∇J(θ;x(i);y(i)) , where {x(i) ,y(i)} are the training examples.


#All optimizers implement a step() method, that updates the parameters. It can be used in two ways:
  #optimizer.step()
#This is a simplified version supported by most optimizers. 
#The function can be called once the gradients are computed using e.g. backward().
#lr_scheduler sets the learning rate of all parameter groups

%timeit model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss #.loss calculates loss of output with given loss function
        loss.backward() #basically does backpropagation. It calculates the gradient of the error function in a NN for each paramter x
        optimizer.step()
        lr_scheduler.step() #calculated from optimizer
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:

#to check the resulst we just use a metric from datasets library

metric= load_metric("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()